In [ ]:
import cv2  # OpenCV 라이브러리로 이미지 처리 기능을 제공
import numpy as np  # 수치 계산을 위한 라이브러리, 배열 연산을 효율적으로 처리
from skimage.metrics import structural_similarity as ssim  # SSIM 계산을 위한 함수 가져오기
from skimage import color  # 이미지 색상 공간 변환을 위한 모듈
import torch  # 딥러닝을 위한 PyTorch 라이브러리
import lpips  # LPIPS 계산을 위한 라이브러리
from PIL import Image  # 이미지 처리 및 파일 입출력을 위한 PIL 라이브러리

def calculate_psnr(img1, img2):
    """두 이미지 사이의 PSNR 값을 계산하는 함수"""
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:  # MSE가 0이면 두 이미지는 동일합니다.
        return float('inf')
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

def calculate_ssim(img1, img2):
    """두 이미지 사이의 SSIM 값을 계산하는 함수"""
    return ssim(img1, img2, win_size=3, channel_axis=2)

def calculate_delta_e(img1, img2):
    """두 이미지 사이의 CIEDE2000 색상 차이를 계산하는 함수"""
    img1_lab = color.rgb2lab(img1)
    img2_lab = color.rgb2lab(img2)
    delta_e = color.deltaE_ciede2000(img1_lab, img2_lab)
    return np.mean(delta_e)

def load_image(image_path, size=(256, 256)):
    """이미지를 로드하고 크기를 조정한 후 텐서로 변환"""
    img = Image.open(image_path).convert('RGB')
    img = img.resize(size)  # 이미지 크기 조정
    img = np.array(img).astype(np.float32) / 255.0
    img = torch.tensor(img).permute(2, 0, 1).unsqueeze(0)  # (H, W, C) -> (C, H, W) -> (1, C, H, W)
    return img

def calculate_lpips(img1_path, img2_path, model):
    """두 이미지 사이의 LPIPS 값을 계산하는 함수"""
    img1 = load_image(img1_path)
    img2 = load_image(img2_path)
    return model(img1, img2).item()

def resize_image_to_match(img1, img2):
    """img1의 크기에 맞게 img2를 리사이즈하는 함수"""
    return cv2.resize(img2, (img1.shape[1], img1.shape[0]))

# 이미지 경로
img1_path = 'your_path'
img2_path = 'your_path'

# 이미지 로드
img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

# 이미지 크기 맞추기
img2_resized = resize_image_to_match(img1, img2)

# PSNR 및 SSIM 계산
psnr_value = calculate_psnr(img1, img2_resized)
ssim_value = calculate_ssim(img1, img2_resized)

print(f"PSNR: {psnr_value} dB")
print(f"SSIM: {ssim_value}")

# ΔE 계산
delta_e_value = calculate_delta_e(img1, img2_resized)
print(f"Mean ΔE: {delta_e_value}")

# LPIPS 모델 로드 (VGG 기반)
lpips_model = lpips.LPIPS(net='vgg')

# LPIPS 계산
lpips_value = calculate_lpips(img1_path, img2_path, lpips_model)
print(f"LPIPS: {lpips_value}")